In [1]:
import rsa_data as rd
import config
import pandas as pd
import numpy as np
import uuid
from io import StringIO
import csv
pd.set_option("display.max_columns", None)
src = r"S:\Michael Brandt\SMEC RSV Files_GP PRM Sites_Dec21toFeb22"
test1 = r"S:\Michael Brandt\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\0006-20211231.RSV"
problem_files = [
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0263-20220228.RSV', 
]

In [2]:
TYPE10_DATA_COLUMN_NAMES = ['site_id', 'header_id', "year", 'number_of_fields_associated_with_the_basic_vehicle_data', 'data_source_code', 'edit_code', 'departure_date', 'departure_time', 'assigned_lane_number', 'physical_lane_number', 'forward_reverse_code', 'vehicle_category', 'vehicle_class_code_primary_scheme', 'vehicle_class_code_secondary_scheme', 'vehicle_speed', 'vehicle_length', 'site_occupancy_time_in_milliseconds', 'chassis_height_code', 'vehicle_following_code', 'vehicle_tag_code', 'trailer_count', 'axle_count', 'bumper_to_1st_axle_spacing', 'tyre_type', 'sub_data_type_code_vx', 'vehicle_registration_number', 'number_of_images', 'image_name_1', 'image_name_2', 'image_name_3', 
'sub_data_type_code_sx', 'number_of_axles_spacings_counted', 'axle_spacing_1_between_individual_axles_cm', 'axle_spacing_2_between_individual_axles_cm', 'axle_spacing_3_between_individual_axles_cm', 'axle_spacing_4_between_individual_axles_cm', 'axle_spacing_5_between_individual_axles_cm', 'axle_spacing_6_between_individual_axles_cm', 'axle_spacing_7_between_individual_axles_cm', 'axle_spacing_8_between_individual_axles_cm',
'start_datetime', 'direction', 'forward_direction_code', 
'sub_data_type_code_wx', 'number_of_wheel_masses', 'offset_sensor_detesction_code', 'mass_measurement_resolution', 'wheel_mass_for_wheel_1', 'wheel_mass_for_wheel_2', 'wheel_mass_for_wheel_3', 'wheel_mass_for_wheel_4', 'wheel_mass_for_wheel_5', 'wheel_mass_for_wheel_6', 'wheel_mass_for_wheel_7', 'wheel_mass_for_wheel_8', 'wheel_mass_for_wheel_9', 'wheel_mass_for_wheel_10'
]

TYPE10_HEADER_COLUMN_NAMES = ['header_id', 'data_description', 'vehicle_classification_scheme_primary', 'vehicle_classification_scheme_secondary', 'maximum_gap_milliseconds', 'maximum_differential_speed'
]

In [3]:
def getfiles(path: str):
    print("COLLECTING FILES......")
    src = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if (
                name.endswith(".RSA")
                or name.endswith(".rsa")
                or name.endswith(".rsv")
                or name.endswith(".RSV")
            ):
                p = os.path.join(root, name)
                src.append(p)
    src = list(set(src))
    return src

In [4]:
def to_df(file: str) -> pd.DataFrame:
    df = pd.read_csv(file, header=None, sep=" ", low_memory=False)
    df = df[0].str.split("\s+|,\s+|,", expand=True)
    df = pd.DataFrame(df)
    return df

def push_to_db(df, table, subset) -> None:
    try:
        df.to_sql(
            table,
            con=config.ENGINE,
            schema="trafc",
            if_exists="append",
            index=False,
            method=psql_insert_copy,
        )
    except Exception:
        df = df.drop_duplicates(subset=subset)
        df.to_sql(
            table,
            con=config.ENGINE,
            schema="trafc",
            if_exists="append",
            index=False,
            method=psql_insert_copy,
        )


def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ", ".join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = "{}.{}".format(table.schema, table.name)
        else:
            table_name = table.name

        sql = "COPY {} ({}) FROM STDIN WITH CSV".format(table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

def join(header: pd.DataFrame, data: pd.DataFrame) -> pd.DataFrame:
    if data.empty:
        df = pd.DataFrame()
    else:
        q = """
		SELECT header.header_id, header.station_name, data.*
		FROM header
		LEFT JOIN data ON data.start_datetime WHERE data.start_datetime >= header.start_datetime AND data.end_datetime <= header.end_datetime;
		"""
        q2 = """UPDATE data set header_id = (SELECT header_id from header WHERE data.start_datetime >= header.start_datetime AND data.counttime_end <= header.enddate)"""
        pysqldf = lambda q: sqldf(q, globals())
        df = sqldf(q, locals())
        df = pd.DataFrame(df)
    return df


def data_join(data: pd.DataFrame, header: pd.DataFrame) -> pd.DataFrame:
    if data is None:
        pass
    elif data.empty:
        pass
    else:
        data = pd.DataFrame(data)
        data = join(header, data)
    return data

def get_direction(lane_number, df: pd.DataFrame) -> pd.DataFrame:
        filt = df[1] == lane_number
        df = df.where(filt)
        df = df[2].dropna()
        df = int(df)
        return df


In [5]:
donelist = [
'S:\Michael Brandt\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\0597-20211231.RSV',
'S:\Michael Brandt\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\0133-20220228.RSV',
'S:\Michael Brandt\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\0133-20220131.RSV',
'S:\Michael Brandt\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\0597-20211231.RSV',
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\n7-20220131.RSV', 
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x013-20220228.RSV', 
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x0b1-20211231.RSV', 
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\n3-20220228.RSV', 
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x009-20220228.RSV', 
'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x073-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x088-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\n3-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\n3-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x1b7-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x193-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\x1a3-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0013-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0131-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0123-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0009-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0073-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0108-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0123-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0123-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0337-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0313-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0323-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0108-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0127-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0313-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0323-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0700-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0006-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0353-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0072-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0350-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0141-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0353-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0009-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0200-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0323-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0400-20220222.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0313-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0013-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0073-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\1699-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0095-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0045-20220104.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0044-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0151-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0141-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0350-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0048-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0131-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0095-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0700-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0072-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0108-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0006-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0006-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0045-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0288-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\1738-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\1699-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0065-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0169-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0169-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0131-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0073-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0141-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0009-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\1738-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0013-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0041-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0353-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0288-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0127-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0161-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0041-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0072-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0065-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0288-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\1699-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0044-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0169-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0065-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0048-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0337-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\1738-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0200-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0350-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0041-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0151-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0337-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0700-20220228.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0151-20211231.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0200-20220131.RSV', 'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0095-20211231.RSV']


In [6]:
def get_head(df) -> pd.DataFrame:
    dfh = pd.DataFrame(
        df.loc[
            (df[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
            | (
                (df[0].isin(["21", "70", "30", "13", "60"]))
                & (~df[1].isin(["0", "1", "2", "3", "4"]))
            )
            | (
                (df[0].isin(["10"]))
                & (df[1].isin(["1", "8", "5", "01", "08", "05"]))
            )
        ]
    ).dropna(axis=1, how="all")
    dfh["index"] = dfh.index
    breaks = dfh["index"].diff() != 1
    groups = breaks.cumsum()
    dfh["newindex"] = groups
    dfh = dfh.set_index("newindex")
    dfh = dfh.drop(columns=["index"])
    return dfh

def headers(dfh: pd.DataFrame) -> pd.DataFrame:
    if not dfh.empty:
        headers = pd.DataFrame()
        headers["site_id"] = dfh.loc[dfh[0] == "S0", 1].astype(str)
        if not dfh.loc[dfh[0] == "S1", 1:].empty:
            headers["station_name"] = (
                dfh.loc[dfh[0] == "S1", 1:]
                .dropna(axis=1)
                .apply(" ".join, axis=1)
                .astype(str)
            )
        else:
            headers["station_name"] = (
                dfh.loc[dfh[0] == "S0", 2:]
                .dropna(axis=1)
                .apply(" ".join, axis=1)
                .astype(str)
            )

        try:
            headers["y"] = dfh.loc[dfh[0] == "S0", 5].astype(float)
            headers["x"] = dfh.loc[dfh[0] == "S0", 6].astype(float)
        except Exception:
            pass

        headers["number_of_lanes"] = dfh.loc[dfh[0] == "L0", 2].astype(int)

        try:
            headers["speedbin1"] = dfh.loc[dfh[0] == "21", 4].astype(int)
            headers["speedbin2"] = dfh.loc[dfh[0] == "21", 5].astype(int)
            headers["speedbin3"] = dfh.loc[dfh[0] == "21", 6].astype(int)
            headers["speedbin4"] = dfh.loc[dfh[0] == "21", 7].astype(int)
            headers["speedbin5"] = dfh.loc[dfh[0] == "21", 8].astype(int)
            headers["speedbin6"] = dfh.loc[dfh[0] == "21", 9].astype(int)
            headers["speedbin7"] = dfh.loc[dfh[0] == "21", 10].astype(int)
            headers["speedbin8"] = dfh.loc[dfh[0] == "21", 11].astype(int)
            headers["speedbin9"] = dfh.loc[dfh[0] == "21", 12].astype(int)
            headers["type_21_count_interval_minutes"] = dfh.loc[
                dfh[0] == "21", 1
            ].astype(int)
            headers["type_21_programmable_rear_to_rear_headway_bin"] = dfh.loc[
                dfh[0] == "21", 3
            ].astype(int)
            headers["type_21_program_id"] = "2"
        except Exception:
            pass
        try:
            headers["type_10_vehicle_classification_scheme_primary"] = dfh.loc[
                dfh[0] == "10", 1
            ].astype(int)
            headers["type_10_vehicle_classification_scheme_secondary"] = dfh.loc[
                dfh[0] == "10", 2
            ].astype(int)
            headers["type_10_maximum_gap_milliseconds"] = dfh.loc[
                dfh[0] == "10", 3
            ].astype(int)
            headers["type_10_maximum_differential_speed"] = dfh.loc[
                dfh[0] == "10", 4
            ].astype(int)
        except Exception:
            pass
        try:
            headers["type_30_summary_interval_minutes"] = dfh.loc[
                dfh[0] == "30", 2
            ].astype(int)
            headers["type_30_vehicle_classification_scheme"] = dfh.loc[
                dfh[0] == "30", 3
            ].astype(int)
        except Exception:
            pass
        try:
            headers["type_70_summary_interval_minutes"] = dfh.loc[
                dfh[0] == "70", 1
            ].astype(int)
            headers["type_70_vehicle_classification_scheme"] = dfh.loc[
                dfh[0] == "70", 2
            ].astype(int)
            headers["type_70_maximum_gap_milliseconds"] = dfh.loc[
                dfh[0] == "70", 3
            ].astype(int)
            headers["type_70_maximum_differential_speed"] = dfh.loc[
                dfh[0] == "70", 4
            ].astype(int)
            headers["type_70_error_bin_code"] = dfh.loc[dfh[0] == "70", 5].astype(
                int
            )
        except Exception:
            pass

        if not dfh.loc[dfh[0] == "D3", 1].empty:
            headers["start_datetime"] = dfh.loc[dfh[0] == "D3", 1].astype(str)
            headers["start_time"] = dfh.loc[dfh[0] == "D3", 2].astype(str)
            headers["end_datetime"] = dfh.loc[dfh[0] == "D3", 3].astype(str)
            headers["end_time"] = dfh.loc[dfh[0] == "D3", 4].astype(str)
        else:
            headers["start_datetime"] = dfh.loc[dfh[0] == "D1", 1].astype(str)
            headers["start_time"] = dfh.loc[dfh[0] == "D1", 2].astype(str)
            headers["end_datetime"] = dfh.loc[dfh[0] == "D1", 3].astype(str)
            headers["end_time"] = dfh.loc[dfh[0] == "D1", 4].astype(str)

        # headers["end_datetime"] = headers.apply(
        #     lambda x: pd.to_datetime(
        #         x["end_datetime"] + x["end_time"], format="%y%m%d%H%M%S"
        #     )
        #     if (
        #         x["end_time"] != "240000"
        #         and len(x["end_datetime"]) == 6
        #         and len(x["end_time"]) == 6
        #     )
        #     else (
        #         pd.to_datetime(
        #             x["end_datetime"] + x["end_time"], format="%y%m%d%H%M%S%f"
        #         )
        #         if (
        #             x["end_time"] != "24000000"
        #             and len(x["end_datetime"]) == 6
        #             and len(x["end_time"]) == 8
        #         )
        #         else (
        #             pd.to_datetime(
        #                 x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M%S"
        #             )
        #             if (
        #                 x["end_time"] != "240000"
        #                 and len(x["end_datetime"]) == 8
        #                 and len(x["end_time"]) == 6
        #             )
        #             else (
        #                 pd.to_datetime(
        #                     x["end_datetime"] + x["end_time"],
        #                     format="%Y%m%d%H%M%S%f",
                        # )
                        # if (
                        #     x["end_time"] != "24000000"
                        #     and len(x["end_datetime"]) == 8
                        #     and len(x["end_time"]) == 8
                        # )
                        # else (
                        #     pd.to_datetime(x["end_datetime"], format="%y%m%d")
                        #     + timedelta(days=1)
                        #     if (
                        #         x["end_time"] == "240000"
                        #         and len(x["end_datetime"]) == 6
                            #     and len(x["end_time"]) == 6
                            # )
                            # else (
                            #     pd.to_datetime(x["end_datetime"], format="%y%m%d")
                            #     + timedelta(days=1)
                            #     if (
                            #         x["end_time"] == "24000000"
                            #         and len(x["end_datetime"]) == 6
                            #         and len(x["end_time"]) == 8
                            #     )
                            #     else (
                            #         pd.to_datetime(
                            #             x["end_datetime"], format="%Y%m%d"
                            #         )
                            #         + timedelta(days=1)
                            #         if (
                            #             x["end_time"] == "240000"
                                #         and len(x["end_datetime"]) == 8
                                #         and len(x["end_time"]) == 6
                                #     )
                                #     else (
                                #         pd.to_datetime(
                                #             x["end_datetime"], format="%Y%m%d"
                                #         )
                                #         + timedelta(days=1)
                                #         if (
                                #             x["end_time"] == "24000000"
                                #             and len(x["end_datetime"]) == 8
                                #             and len(x["end_time"]) == 8
                                #         )
                                #         else pd.to_datetime(
                                #             x["end_datetime"] + x["end_time"]
                                #         )
                                #     )
                                # )
        #                     )
        #                 )
        #             )
        #         )
        #     ),
        #     axis=1,
        # )

        # headers["start_datetime"] = headers.apply(
        #     lambda x: pd.to_datetime(
        #         x["start_datetime"] + x["start_time"], format="%y%m%d%H%M%S"
        #     )
        #     if (
        #         x["start_time"] != "240000"
        #         and len(x["start_datetime"]) == 6
        #         and len(x["start_time"]) == 6
        #     )
        #     else (
        #         pd.to_datetime(
        #             x["start_datetime"] + x["start_time"], format="%y%m%d%H%M%S%f"
        #         )
        #         if (
        #             x["start_time"] != "24000000"
        #             and len(x["start_datetime"]) == 6
        #             and len(x["start_time"]) == 8
        #         )
        #         else (
        #             pd.to_datetime(
        #                 x["start_datetime"] + x["start_time"], format="%Y%m%d%H%M%S"
        #             )
        #             if (
        #                 x["start_time"] != "240000"
        #                 and len(x["start_datetime"]) == 8
                    #     and len(x["start_time"]) == 6
                    # )
                    # else (
                    #     pd.to_datetime(
                    #         x["start_datetime"] + x["start_time"],
                    #         format="%Y%m%d%H%M%S%f",
                    #     )
                    #     if (
                    #         x["start_time"] != "24000000"
                    #         and len(x["start_datetime"]) == 8
                    #         and len(x["start_time"]) == 8
                    #     )
                    #     else (
                    #         pd.to_datetime(x["start_datetime"], format="%y%m%d")
                    #         + timedelta(days=1)
                    #         if (
                    #             x["start_time"] == "240000"
                    #             and len(x["start_datetime"]) == 6
                    #             and len(x["start_time"]) == 6
                    #         )
                            # else (
                            #     pd.to_datetime(x["start_datetime"], format="%y%m%d")
                            #     + timedelta(days=1)
                            #     if (
                            #         x["start_time"] == "24000000"
                            #         and len(x["start_datetime"]) == 6
                            #         and len(x["start_time"]) == 8
                            #     )
                            #     else (
                            #         pd.to_datetime(
                            #             x["start_datetime"], format="%Y%m%d"
                            #         )
                            #         + timedelta(days=1)
                            #         if (
                            #             x["start_time"] == "240000"
                            #             and len(x["start_datetime"]) == 8
                            #             and len(x["start_time"]) == 6
                            #         )
        #                             else (
        #                                 pd.to_datetime(
        #                                     x["start_datetime"], format="%Y%m%d"
        #                                 )
        #                                 + timedelta(days=1)
        #                                 if (
        #                                     x["start_time"] == "24000000"
        #                                     and len(x["start_datetime"]) == 8
        #                                     and len(x["start_time"]) == 8
        #                                 )
        #                                 else pd.to_datetime(
        #                                     x["start_datetime"] + x["start_time"]
        #                                 )
        #                             )
        #                         )
        #                     )
        #                 )
        #             )
        #         )
        #     ),
        #     axis=1,
        # )

        # headers = headers.drop(["start_time"], axis=1)
        # headers = headers.drop(["end_time"], axis=1)

        # headers["start_datetime"] = pd.to_datetime(headers["start_datetime"])
        # headers["end_datetime"] = pd.to_datetime(headers["end_datetime"])
        headers["site_id"] = headers["site_id"].astype(str)

        try:
            headers["instrumentation_description"] = (
                dfh.loc[dfh[0] == "I0", 1:]
                .dropna(axis=1)
                .apply(" ".join, axis=1)
                .astype(str)
            )
        except Exception:
            headers["instrumentation_description"] = None

        try:
            headers["type_30_summary_interval_minutes"] = headers[
                "type_21_count_interval_minutes"
            ]
        except Exception:
            pass
        try:
            headers["type_70_summary_interval_minutes"] = headers[
                "type_21_count_interval_minutes"
            ]
            headers["type_70_vehicle_classification_scheme"] = headers[
                "type_21_count_interval_minutes"
            ]
            headers["type_70_vehicle_classification_scheme"] = headers[
                "type_21_count_interval_minutes"
            ]
        except Exception:
            pass

        headers = headers.fillna(method="ffill")
        headers = headers.fillna(method="bfill")

        headers = headers.drop_duplicates(ignore_index=True)

        headers["header_id"] = ""
        headers["header_id"] = headers["header_id"].apply(
            lambda x: str(uuid.uuid4())
        )

    else:
        pass
    return headers


In [7]:
df = to_df(r'S:\\Michael Brandt\\SMEC RSV Files_GP PRM Sites_Dec21toFeb22\\0263-20220228.RSV')

In [8]:
TYPE10_DATA_COLUMN_NAMES = [
"data_type_code",
"number_of_fields_associated_with_the_basic_vehicle_data",
"data_source_code",
"edit_code",
"departure_date",
"departure_time",
"assigned_lane_number",
"physical_lane_number",
"forward_reverse_code",
"vehicle_category",
"vehicle_class_code_primary_scheme",
"vehicle_class_code_secondary_scheme",
"vehicle_speed",
"vehicle_length",
"site_occupancy_time_in_milliseconds",
"chassis_height_code",
"vehicle_following_code",
"vehicle_tag_code",
"trailer_count",
"axle_count",
"bumper_to_1st_axle_spacing",
"tyre_type"  
]

In [58]:
data = df.loc[(df[0] == "10") & (df[3].isin(["1", "0"]))].dropna(
    axis=1, how="all"
)
dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
    axis=1, how="all"
)

In [86]:
if data.empty:
    print("data empty")
    print(data)
else:
    num_of_fields = int(data.iloc[:,1].unique()[0])
    ddf = data.iloc[:,: 2 + num_of_fields]
    ddf.reset_index(inplace=True)

    cols = ['index']
    for i in range(ddf.shape[1]-1):
        cols.append(config.TYPE10_DATA_COLUMN_NAMES[i])
    ddf = pd.DataFrame(ddf.values, columns=cols)
    ddf["data_id"] = ddf.apply(lambda x: uuid.uuid4(), axis=1)

    if data.shape[1] > ddf.shape[1]:
        sub_data_df = pd.DataFrame(columns=['index','sub_data_type_code','offset_sensor_detection_code','mass_measurement_resolution_kg', 'number','value'])
        for index, row in data.iterrows():
            col = int(row[1]) + 2
            while col < len(row) and row[col] != None:
                sub_data_type = row[col]
                col += 1
                NoOfType = int(row[col])        
                col +=1
                if sub_data_type[0].lower() in ['w','a','g']:
                    odc = row[col]
                    col += 1
                    mmr = row[col]
                    col +=1
                    for i in range(0,NoOfType):
                        tempdf = pd.DataFrame([[index,
                        sub_data_type,
                        odc,
                        mmr,
                        i + 1,
                        row[col]]
                        ], columns = ['index',
                        'sub_data_type_code',
                        'offset_sensor_detection_code',
                        'mass_measurement_resolution_kg',
                        'number',
                        'value'
                        ])
                        sub_data_df = pd.concat([sub_data_df, tempdf])
                        col += 1
                else:
                    for i in range(0,NoOfType):
                        tempdf = pd.DataFrame([[index, 
                        sub_data_type,
                        i + 1,
                        row[col]]], columns = ['index' ,
                        'sub_data_type_code',
                        'number',
                        'value'])
                        sub_data_df = pd.concat([sub_data_df, tempdf])
                        col += 1

    sub_data_df = sub_data_df.merge(ddf[['index', 'id']], how='left', on='index')


In [87]:
ddf = ddf.fillna(0)
ddf["assigned_lane_number"] = ddf["assigned_lane_number"].astype(int)
max_lanes = ddf["assigned_lane_number"].max()
try:
    ddf["direction"] = ddf.apply(
    lambda x: "P" if x["assigned_lane_number"] <= (int(max_lanes) / 2) else "N",
    axis=1,
)
    direction = dfh2.loc[dfh2[0] == "L1", 1:3]
    direction = direction.drop_duplicates()
except:
    pass

if ddf["departure_date"].map(len).isin([8]).all():
    ddf["start_datetime"] = pd.to_datetime(
        ddf["departure_date"] + ddf["departure_time"],
        format="%Y%m%d%H%M%S%f",
    )
elif ddf["departure_date"].map(len).isin([6]).all():
    ddf["start_datetime"] = pd.to_datetime(
        ddf["departure_date"] + ddf["departure_time"],
        format="%y%m%d%H%M%S%f",
    )
ddf['year'] = ddf['start_datetime'].dt.year
t1 = dfh2.loc[dfh2[0] == "S0", 1].unique()
ddf["site_id"] = str(t1[0])
ddf["site_id"] = ddf["site_id"].astype(str)
ddf['departure_time'] = pd.to_datetime(ddf['departure_time'], format='%H%M%S%f')

ddf = ddf.drop_duplicates()
ddf["start_datetime"] = ddf["start_datetime"].astype("datetime64[ns]")

In [97]:
ddf = ddf.replace(r'^\s*$', np.NaN, regex=True)

scols = ddf.select_dtypes('object').columns

ddf[scols] = ddf[scols].apply(pd.to_numeric, axis=1, errors='ignore')

In [93]:
TYPE10_DATA_COL_LIST = ["data_id",
"site_id",
"header_id",
"year",
"start_datetime",
"direction", 
"number_of_fields_associated_with_the_basic_vehicle_data",
"data_source_code", 
"edit_code", 
"departure_date", 
"departure_time", 
"assigned_lane_number",
"physical_lane_number",
"forward_reverse_code",
"vehicle_category",
"vehicle_class_code_primary_scheme",
"vehicle_class_code_secondary_scheme",
"vehicle_speed",
"vehicle_length",
"site_occupancy_time_in_milliseconds",
"chassis_height_code",
"vehicle_following_code",
"vehicle_tag_code",
"trailer_count",
"axle_count",
"bumper_to_1st_axle_spacing",
"tyre_type"]

In [94]:
ddf = ddf[ddf.columns.intersection(TYPE10_DATA_COL_LIST)]

In [98]:
ddf.to_sql(
    "electronic_count_data_type_10",
    con=config.ENGINE,
    schema="trafc",
    if_exists="append",
    index=False,
    # method=psql_insert_copy,
)

946

In [116]:
sub_data_df = sub_data_df.replace(r'^\s*$', np.NaN, regex=True)
sub_data_df = sub_data_df.drop("index", axis=1)

In [117]:
wx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'w']
sx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 's']
gx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'g']
vx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'v']
tx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 't']
ax_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'a']
cx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'c']

In [122]:
wx_data.rename(columns = {"value":"wheel_mass", "number":"wheel_mass_number", "id":"type10_id"}, inplace=True)
sx_data.rename(columns = {"value":"axle_spacing_cm", "number":"axle_spacing_number", "id":"type10_id"}, inplace=True)
sx_data = sx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)

C:\Users\MB2705851\AppData\Local\Temp\ipykernel_13148\3861411644.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wx_data.rename(columns = {"value":"wheel_mass", "number":"wheel_mass_number", "id":"type10_id"}, inplace=True)
C:\Users\MB2705851\AppData\Local\Temp\ipykernel_13148\3861411644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sx_data.rename(columns = {"value":"axle_spacing_cm", "number":"axle_spacing_number", "id":"type10_id"}, inplace=True)


In [112]:
wx_data = wx_data.replace(r'^\s*$', np.NaN, regex=True)

In [113]:
if wx_data.empty:
    pass
else:
    wx_data.to_sql(
    "traffic_e_type10_wheel_mass",
    con=config.ENGINE,
    schema="trafc",
    if_exists="append",
    index=False,
    method=psql_insert_copy,
)

In [123]:
if sx_data.empty:
    pass
else:
    sx_data.to_sql(
    "traffic_e_type10_axle_spacing",
    con=config.ENGINE,
    schema="trafc",
    if_exists="append",
    index=False,
    method=psql_insert_copy,
)